# NanoPyx "Codeless" Jupyter Notebook

This notebook allows you to perform denoising on either 2 or 3 dimensional arrays.

To use this notebook you don't need to interact with any code, just run cells in order and a graphical user interface will pop-up showcasing the parameters for each step.
When running this notebook on Colab, it will automatically prompt you to connect to your google Drive, from where you can select the input data.
To save the output of this notebook, make sure to select the "save output" option. Output will be saved on the same directory as the loaded image. If using an example dataset it will be saved on the path Python is currently running (if locally running the notebook) or on Colab files (if running on Colab, files button on the left bar).
If you found this work useful for your research please consider citing our [preprint](https://www.biorxiv.org/content/10.1101/2023.08.13.553080v1)  and out Github repo [![DOI](https://zenodo.org/badge/505388398.svg)](https://zenodo.org/badge/latestdoi/505388398)

A. Buades, B. Coll, & J-M. Morel. A non-local algorithm for image denoising. In CVPR 2005, Vol. 2, pp. 60-65, IEEE. DOI:10.1109/CVPR.2005.38
Jacques Froment. Parameter-Free Fast Pixelwise Non-Local Means Denoising. Image Processing On Line, 2014, vol. 4, pp. 300-326. DOI:10.5201/ipol.2014.120




In [ ]:
#@title Fix numpy version in Google Colab, can be skipped if running in local environment. Session will now restart automatically. You can then proceed to the next cell.
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install -q numpy==1.26.4
    !pip install -q mako==1.3.0

    print("Session will now restart automatically. You can then proceed to the next cell.")
    import os
    os.kill(os.getpid(), 9)




In [ ]:
#@title Install NanoPyx, import necessary libraries and connect to Google Drive
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install -q "nanopyx[colab]"
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
    drive.mount('/content/drive')
else:
    !pip install -q "nanopyx[jupyter]"

import io
import os
import cv2 as cv
import skimage
import nanopyx
import stackview
import numpy as np
import tifffile as tiff
import matplotlib as mpl
import ipywidgets as widgets
from PIL import Image
from IPython.display import display, clear_output
from matplotlib import pyplot as plt

try:
    from ezinput import EZInput as EasyGui
except ImportError:
    from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.core.utils.find_files import find_files
from nanopyx.data.download import ExampleDataManager

cwd = os.getcwd()
image_folder = "datasets"
image_files = []
EDM = ExampleDataManager()
example_datasets = EDM.list_datasets()

_path = os.path.join("..", image_folder)
if os.path.exists(_path):
    image_files += find_files(_path, ".tif")
if os.path.exists(image_folder):
    image_files += find_files(image_folder, ".tif")
image_files += ["Example dataset: "+dataset for dataset in example_datasets]




In [ ]:
#@title Load image stack
# Create a GUI
gui_data = EasyGui("Data Loader")
global own_data
own_data = True

def on_button_select_own(b):
    clear_output()
    gui_data.add_label(value="Select data to use:")
    gui_data.add_file_upload("upload")
    gui_data.add_dropdown("cmaps", description="Colormap:",
                          options=sorted(list(mpl.colormaps)),
                          value="viridis", remember_value=True)
    gui_data.add_button("load_data_own", description="Load data")
    gui_data["load_data_own"].on_click(on_button_load_data_clicked)
    gui_data.show()

def on_button_select_example(b):
    clear_output()
    gui_data.add_label(value="Select data to use:")
    gui_data.add_dropdown("data_source", options=image_files,
                    value="Example dataset: "+example_datasets[4], remember_value=True)
    gui_data.add_dropdown("cmaps", description="Colormap:",
                          options=sorted(list(mpl.colormaps)),
                          value="viridis", remember_value=True)
    gui_data.add_button("load_data", description="Load data")
    gui_data["load_data"].on_click(on_button_load_data_clicked_example)
    gui_data.show()

def on_button_load_data_clicked(b):
    clear_output()
    gui_data.show()
    global dataset_original
    global own_data
    own_data = True
    # disable button
    gui_data["load_data_own"].disabled = True
    gui_data["load_data_own"].description = "Loading..."
    dataset_original = tiff.imread(gui_data["upload"].selected)
    gui_data["load_data_own"].disabled = False
    gui_data["load_data_own"].description = "Load data"
    gui_data._main_display.children = gui_data._main_display.children + (stackview.slice(dataset_original, colormap=gui_data["cmaps"].value, continuous_update=True),)

def on_button_load_data_clicked_example(b):
    clear_output()
    gui_data.show()
    global dataset_original
    global own_data
    own_data = False
    # disable button
    gui_data["load_data"].disabled = True
    gui_data["load_data"].description = "Loading..."

    if gui_data["data_source"].value.startswith("Example dataset: "):
        dataset_name = gui_data["data_source"].value.replace(
            "Example dataset: ", "")
        dataset_original = EDM.get_ZipTiffIterator(dataset_name, as_ndarray=True)
        gui_data._main_display.children = gui_data._main_display.children + (stackview.slice(dataset_original, continuous_update=True, colormap=gui_data["cmaps"].value),)
    else:
        dataset_original = skimage.io.imread(gui_data["data_source"].value)
        gui_data._main_display.children = gui_data._main_display.children + (stackview.slice(dataset_original, continuous_update=True, colormap=gui_data["cmaps"].value),)

    # enable button
    gui_data["load_data"].disabled = False
    gui_data["load_data"].description = "Load data"
    gui_data.save_settings()

gui_data.add_button("use_own_data", description="Use Own data")
gui_data["use_own_data"].on_click(on_button_select_own)
gui_data.add_button("use_example_data", description="Use Example data")
gui_data["use_example_data"].on_click(on_button_select_example)
gui_data.show()




# Use Non-local means denoising on selected data
# NLM Denoising Parameters:

- **Patch size:** Size of patches used for denoising.
- **Patch distance:** Maximal pixel distance to search patches used for denoising.
- **h:** Cut-off distance (in gray levels). A higher h results in a smoother image, at the expense of blurring features. For a Gaussian noise of standard deviation sigma, a rule of thumb is to choose the value of h to be sigma of slightly less.
- **sigma:** The standard deviation of the Gaussian noise.




In [ ]:
#@title Create NLM Denoising GUI
gui_nlm = EasyGui("nlm")
from nanopyx.core.transform._le_nlm_denoising import NLMDenoising

def run_nlm(b):
    clear_output()
    gui_nlm.show()
    gui_nlm.save_settings()
    patch_size = gui_nlm["patch_size"].value
    patch_distance = gui_nlm["patch_distance"].value
    h = gui_nlm["h"].value
    sigma = gui_nlm["sigma"].value
    # disable button while running
    gui_nlm["run"].disabled = True
    gui_nlm["run"].description = "Running..."

    denoiser = NLMDenoising()

    global dataset_nlm
    dataset_nlm = denoiser.run(dataset_original, patch_size=patch_size, patch_distance=patch_distance, h=h, sigma=sigma)
    # enable button again
    gui_nlm["run"].disabled = False
    gui_nlm["run"].description = "Run"
    if gui_nlm["save"].value:
        if own_data:
            path = gui_data["upload"].selected_path
            name = gui_data["upload"].selected_filename.split(".")[0]
            tiff.imwrite(path + os.sep + name + "_nlm_denoised.tif", dataset_nlm)
        else:
            name = gui_data["data_source"].value.replace("Example dataset: ", "")
            tiff.imwrite(name + "_nlm_denoised.tif", dataset_nlm)
    gui_nlm._main_display.children = gui_nlm._main_display.children + (stackview.slice(dataset_nlm, colormap=gui_nlm["cmaps"].value, continuous_update=True),)

gui_nlm.add_int_slider("patch_size", description="Patch Size", min=1, max=dataset_original.shape[-1]//2, value=5)
gui_nlm.add_int_slider("patch_distance", description="Patch Distance", min=1, max=dataset_original.shape[-1]//2, value=10)
gui_nlm.add_float_text("h", description="h", value=0.1, remember_value=True)
gui_nlm.add_float_text("sigma", description="sigma", value=0.1, remember_value=True)
gui_nlm.add_checkbox("save", description="Save Output", value=True)
gui_nlm.add_dropdown("cmaps", description="Colormap:",
                      options=sorted(list(mpl.colormaps)),
                      value="viridis", remember_value=True)
gui_nlm.add_button("run", description="Run")
gui_nlm['run'].on_click(run_nlm)
gui_nlm.show()

